# Starting EDA of pH project

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [81]:
csv_file = "../Data/R12-over-10-copies.csv"
df = pd.read_csv(csv_file)
df.head()

,Sequence,Count
0,AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAA...,91
1,AGTGCCATCGTGCGTATCCTTCACGTTTGTTCTACAAAAGCGAAGC...,86
2,AGTGCCATCGTGCGTATCCTGAACATTGACGTGTAGTCAATGCGAA...,83
3,AGTGCCATCGTGCGTATCCCGCTCCGCTAATATCTATGTTAGCGAA...,80
4,AGTGCCATCGTGCGTATCCTGAACATAGACGTTTAGTCTATGCGAA...,78


In [82]:
df.shape

(10383, 2)

In [83]:
sum(df.iloc[:, 0].isnull())

0

In [84]:
sum(df.iloc[:, 1].isnull())

0

In [85]:
df.dtypes

Sequence    object
Count        int64
dtype: object

In [86]:
# check base type proportions
def base_count(text):
    return dict(Counter(text))   

In [87]:
df['Base Counts'] = df['Sequence'].apply(base_count)
# df['Base Counts'][1]['A']
df['A'] = df['Base Counts'].astype(object).apply(lambda x: x.get('A', np.nan))
df['C'] = df['Base Counts'].astype(object).apply(lambda x: x.get('C', np.nan))
df['T'] = df['Base Counts'].astype(object).apply(lambda x: x.get('G', np.nan))
df['G'] = df['Base Counts'].astype(object).apply(lambda x: x.get('T', np.nan))

In [88]:
df.head().iloc[:, 3:]

,A,C,T,G
0,24,26,26,22
1,22,27,29,20
2,23,23,29,23
3,24,25,28,21
4,24,21,30,23


In [89]:
df['Length'] = df['Sequence'].apply(len)

In [90]:
sum((np.sum(df[['A', 'C', 'T', 'G']], axis=1) == df['Length']).isnull()) # Checks base counts add up to length of sequence

0

In [91]:
allowed_keys = {'A', 'C', 'T', 'G'} # Checks that there are no weird nonstandard bases
df['Nonstandard base'] = df['Base Counts'].astype(object).apply(lambda x: not set(x.keys()).issubset(allowed_keys))
np.sum(df['Nonstandard base'])

0

In [92]:
df.head()

,Sequence,Count,Base Counts,A,C,T,G,Length,Nonstandard base
0,AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAA...,91,"{'A': 24, 'G': 26, 'T': 22, 'C': 26}",24,26,26,22,98,False
1,AGTGCCATCGTGCGTATCCTTCACGTTTGTTCTACAAAAGCGAAGC...,86,"{'A': 22, 'G': 29, 'T': 20, 'C': 27}",22,27,29,20,98,False
2,AGTGCCATCGTGCGTATCCTGAACATTGACGTGTAGTCAATGCGAA...,83,"{'A': 23, 'G': 29, 'T': 23, 'C': 23}",23,23,29,23,98,False
3,AGTGCCATCGTGCGTATCCCGCTCCGCTAATATCTATGTTAGCGAA...,80,"{'A': 24, 'G': 28, 'T': 21, 'C': 25}",24,25,28,21,98,False
4,AGTGCCATCGTGCGTATCCTGAACATAGACGTTTAGTCTATGCGAA...,78,"{'A': 24, 'G': 30, 'T': 23, 'C': 21}",24,21,30,23,98,False


In [93]:
df['Length'].value_counts().sort_values(ascending=False)

Length
98     9893
97      325
99       65
96       42
95       11
94        7
88        7
87        4
91        4
84        3
90        3
93        3
86        3
85        3
92        3
89        1
107       1
102       1
108       1
82        1
100       1
83        1
Name: count, dtype: int64

The sequences range in length from 82 to 108, but most commonly at 98.
As I recally, NGS clips the first few base pairs down from nominal 100 because of poor quality.
The other variants in sequence length are likely due to mutations from PCR errors.

In [94]:
v_df = df # Save a verbose version of df
df = df[['Sequence', 'Length', 'Count', 'A', 'C', 'T', 'G']]
df.head()

,Sequence,Length,Count,A,C,T,G
0,AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAA...,98,91,24,26,26,22
1,AGTGCCATCGTGCGTATCCTTCACGTTTGTTCTACAAAAGCGAAGC...,98,86,22,27,29,20
2,AGTGCCATCGTGCGTATCCTGAACATTGACGTGTAGTCAATGCGAA...,98,83,23,23,29,23
3,AGTGCCATCGTGCGTATCCCGCTCCGCTAATATCTATGTTAGCGAA...,98,80,24,25,28,21
4,AGTGCCATCGTGCGTATCCTGAACATAGACGTTTAGTCTATGCGAA...,98,78,24,21,30,23


In [95]:
for col in ['A', 'C', 'T', 'G']:
    df.loc[:, col] = df[col]/df['Length']

In [96]:
df.head()

,Sequence,Length,Count,A,C,T,G
0,AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAA...,98,91,0.244898,0.265306,0.265306,0.224490
1,AGTGCCATCGTGCGTATCCTTCACGTTTGTTCTACAAAAGCGAAGC...,98,86,0.224490,0.275510,0.295918,0.204082
2,AGTGCCATCGTGCGTATCCTGAACATTGACGTGTAGTCAATGCGAA...,98,83,0.234694,0.234694,0.295918,0.234694
3,AGTGCCATCGTGCGTATCCCGCTCCGCTAATATCTATGTTAGCGAA...,98,80,0.244898,0.255102,0.285714,0.214286
4,AGTGCCATCGTGCGTATCCTGAACATAGACGTTTAGTCTATGCGAA...,98,78,0.244898,0.214286,0.306122,0.234694


In [99]:
df['Count'].describe()

count    10383.000000
mean        14.926900
std          6.640851
min         10.000000
25%         11.000000
50%         13.000000
75%         16.000000
max         91.000000
Name: Count, dtype: float64

Examine the base makeup of different count frequency

In [101]:
round(np.mean(df[df['Count'] < 11]['A']), 4)

0.2471

In [105]:
round(np.mean(df[(df['Count'] >= 11) & (df['Count'] < 13)]['A']), 4)

0.2466

In [106]:
round(np.mean(df[(df['Count'] >= 13) & (df['Count'] < 16)]['A']), 4)

0.2464

In [107]:
round(np.mean(df[df['Count'] >= 16 ]['A']), 4)

0.2442

In [108]:
round(np.mean(df[df['Count'] >= 16 ]['C']), 4)

0.2639

In [109]:
round(np.mean(df[df['Count'] >= 16 ]['T']), 4)

0.2702

In [110]:
round(np.mean(df[df['Count'] >= 16 ]['G']), 4)

0.2217